## 2. Answer following questions:


###### 1. Why do we need machine learning methods instead of creating a complicated formula?

复杂的公式往往只能够解决某一种或者某一类问题，可扩展性低维护成本高。

###### 2.  Wha't's the disadvantages of `the 1st Random Choosen` methods in our course? 

随机参数取值变化大小方向都不固定，最终导致的结果可能是不收敛的，无法得到合理的值。并且random choice人为设定的取值范围未必合理

###### 3. Is the `2nd method supervised direction` better than 1st one?  What's the disadvantages of `the 2nd supversied directin` method? 

课程中第二种方法在第一种方式情况下增加了方向监控，每次loss值减小时控制其沿着此方向继续进行，相比于第一种方式执行速度更快每次都是有针对性的执行。
但是第二种方式，只能一定范围内找到其相对较好的值，而无法确定全局最优或极优解

###### 4. Why do we use `Derivative / Gredient` to fit a target function? 

梯度下降能够精确的定位损失函数较小的方向，只要控制好其学习率便能够较快的得到一个最优解

###### 5. In the words 'Gredient Descent', what's the `Gredient` and what's the `Descent`?

在求参数k的值时，往往是通过找到损失函数的最优解。

导数：导数的意义值的是曲线或曲面某点切线的斜率，

方向导数：指导数的变化方向

根据如果导数值大于0，向其反方向不断的迭代更新便能够找到全局或者局部的最优解

###### 6. What's the advantages of `the 3rd gradient descent method` compared to the previous methods?

In [ ]:
梯度下降以数学为基础，能够快速准确的找到最优解。

###### 7. Using the simple words to describe: What's the machine leanring.

机器学习是基于数学的数据驱动的计算机算法

#

# coding 部分

In [5]:
import re
import json, requests
from collections import defaultdict
from operator import itemgetter

##### 获取地铁站，及地铁线

In [48]:
def deal_subway_data():
    with open('beijing.xml') as f:
        datas = f.read()
    pattern = re.compile('\s+lb="(\w+)"')
    station = re.findall(pattern, datas)
    pattern1 = re.compile('<l lid="\w+"\s+lb="(\w+)"')
    lines = re.findall(pattern1, datas)
    return station, lines

##### 获取每条线的 所有站

In [49]:
def subways_related(lines, station):
    subways = {}
    for i in range(len(lines)):
        if lines[i] == '机场线':
            pass
            # subways[lines[i]] = station[station.index(lines[i])+1:]
        else:
            subways[lines[i]] = station[station.index(lines[i])+1:station.index(lines[i+1])]
    subways['机场线'] = ['东直门', '三元桥', '3号航站楼', '2号航站楼']
    return subways

##### 建立地铁线路的关系图

In [50]:
def graph_subway_lines(subways):
    # 地铁线关系图
    subway_line = defaultdict(list)
    for s1 in subways:
        for s2 in subways:
            if s1 == s2: continue
            if len(subways[s1][1:] + subways[s2][1:]) != len(set(subways[s1][1:] + subways[s2][1:])):
                subway_line[s1] = subway_line[s1] + [s2]
    return subway_line

##### 建立地铁站的关系图

In [51]:

def graph_subway_station(subways):
    # 地铁站关系图
    subway_station = {}
    for sub, vals in subways.items():
        for i, v in enumerate(vals):
            if v not in subway_station:
                if i == 0:
                    subway_station[v] = [vals[i + 1]]
                elif i == (len(vals) - 1):
                    subway_station[v] = [vals[i - 1]]
                else:
                    subway_station[v] = [vals[i - 1], vals[i + 1]]
            else:
                if i == 0:
                    subway_station[v] = subway_station[v] + [vals[i + 1]]
                elif i == (len(vals) - 1):
                    subway_station[v] = subway_station[v] + [vals[i - 1]]
                else:
                    subway_station[v] = subway_station[v] + [vals[i - 1], vals[i + 1]]

    return subway_station

##### 根据建立好的地铁线路关系图筛选 优先路线

In [52]:
def check_better_lines(start_to_stop, all_lines):
    """筛选最优路线"""
    better_lines = []
    all_lines = sorted(all_lines, key=lambda s: start_to_stop[0])
    for ss in start_to_stop:
        best_line = None
        for line in all_lines:
            if line[0] == ss[0] and line[-1] == ss[-1]:
                if not best_line:
                    best_line = line
                elif len(line) < len(best_line):
                    best_line = line
                elif len(line) == len(best_line):
                    better_lines.append(best_line)
                    best_line = line
        if best_line:
            better_lines.append(best_line)
    return better_lines

##### 搜索并筛选最优换成线图(以每条地铁线为维度)

In [71]:
def search_lines_graph(start_station_line, stop_station_line, subway_line):
    """搜索并筛选最优换成线图(以每条地铁线为维度)"""
    start_to_stop, all_lines = [], []
    for start_line in start_station_line:
        for stop_line in stop_station_line:
            start_to_stop.append([start_line, stop_line])
            if start_line == stop_line:
                return [[start_line, stop_line]]

    print('可能开始和结束地铁线：{}'.format(start_to_stop), end='\n\n', sep='\n')
    for line in start_to_stop:
        pathes = [[line[0]]]
        seened = []
        while pathes:
            path = pathes.pop(0)
            station = path[-1]
            if station in seened and station == line[1]: continue  # 记录访问过的站，并且是终点站时不继续执行
            for successor in subway_line[station]:
                if successor in seened: continue
                if successor not in start_station_line:  # 例如开始站为”三元桥“换乘站时出现 “机场线/10好像/13好像”这种不正确的组合方式
                    new_path = path + [successor]
                    pathes.append(new_path)
                    if successor == line[1]:
                        if new_path not in all_lines:
                            all_lines.append(new_path)
            seened.append(station)
    better_lines = check_better_lines(start_to_stop, all_lines)
    return better_lines


##### 按照每一站进一步筛选最优路线

In [65]:
def search_station_graph(start, stop, better_lines, subways):
    """按照每一站进一步筛选最优路线"""
    best_lines = []
    for line in better_lines:
        subway = {key: subways[key] for key in line}
        station_graph = graph_subway_station(subway)
        pathes = [[start]]
        seened = []
        while pathes:
            path = pathes.pop(0)
            station = path[-1]
            
            if station in seened and station == stop: continue
            for successor in station_graph[station]:
                if station == stop:
                    break
                if successor in seened: continue
                new_path = path + [successor]
                pathes.append(new_path)
                if successor == stop:
                    best_lines.append({'/'.join(line): new_path})
            if station != stop:  # 终点站访问过后不加入seened，防止查找线路减少
                seened.append(station)
    return best_lines

##### 搜索

In [55]:
def search(start, stop, subways, subway_line, subway_station, sortfunction=None):
    """搜索返回最优路线"""
    start_station_line = []
    stop_station_line = []
    for subway_key, subway_values in subways.items():
        if start in subway_values:
            start_station_line.append(subway_key)
        if stop in subway_values:
            stop_station_line.append(subway_key)

    better_lines = search_lines_graph(start_station_line, stop_station_line, subway_line)

    best_lines = search_station_graph(start, stop, better_lines, subways)

    return sortfunction(best_lines)  if sortfunction else best_lines

##### 排序

In [56]:
# def shortest_transfer_sort(best_lines):
#     mid_dict = {key: value for x in best_lines for key, value in x.items()}
#     mid_list = sorted(mid_dict.items(), key=lambda x: len(x[1]))
#     best_lines = [{x[0]: x[1]} for x in mid_list]
#     return best_lines

In [61]:
def main(start, stop, sortfunction=None):
    station, lines = deal_subway_data()
    subways = subways_related(lines, station) # 每条线对应的地铁站
    subway_line = graph_subway_lines(subways) # 每条地铁线的关系图
    subway_station = graph_subway_station(subways) # 每一站的换乘关系图
    best_lines = search(start, stop, subways, subway_line, subway_station, sortfunction)
    return best_lines

In [75]:
for i in main('三元桥', '西二旗'):
    print('-'*50 +'输出结果⬇️⬇️⬇️' + '-'*50, end='\n\n')
    print(i)
    print('\n')

可能开始和结束地铁线：[['10号线', '13号线'], ['10号线', '昌平线'], ['机场线', '13号线'], ['机场线', '昌平线']]

--------------------------------------------------输出结果⬇️⬇️⬇️--------------------------------------------------

{'10号线/13号线': ['三元桥', '太阳宫', '芍药居', '望京西', '北苑', '立水桥', '霍营', '回龙观', '龙泽', '西二旗']}


--------------------------------------------------输出结果⬇️⬇️⬇️--------------------------------------------------

{'10号线/13号线': ['三元桥', '太阳宫', '芍药居', '惠新西街南口', '安贞门', '北土城', '健德门', '牡丹园', '西土城', '知春路', '五道口', '上地', '西二旗']}


